In [1]:
from datasets import Dataset, DatasetDict
import json
import pandas as pd
import torch
import random
from peft import LoraConfig, TaskType, get_peft_model
from transformers import AutoTokenizer, DataCollatorForSeq2Seq, DataCollatorForLanguageModeling, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer, GenerationConfig

In [2]:
# Open the JSONL file
train_list =[]
# with open('sample.jsonl', 'r') as file:
with open('train.jsonl', 'r') as file:
	for line in file:
		# Parse each line as JSON
		json_obj = json.loads(line)
		
		# Process the JSON object (example: print it)
		# print(json_obj)
		train_list.append(json_obj['text'])

# print('train_list', train_list)
print('train_list[0]', train_list[0:2])
print(len(train_list))

test_index_list = random.sample(range(0, len(train_list)-1), 100)
print(test_index_list)

test_list = [train_list[index] for index in test_index_list]
print(len(test_list))

for example in test_list:
    train_list.remove(example)
    
data = DatasetDict({'train':Dataset.from_dict({"intstruction":train_list}), 'test':Dataset.from_dict({"intstruction":test_list})})
# data = Dataset.from_dict({"intstruction":train_list})
print(data)

train_list[0] ['<|im_start|>system\n你是一个 bilibili 或者 B 站的用户, 你的任务是回复网友的问题<|im_end|>\n<|im_start|>user\n"阿猛，如果你要cos一个角色，你会选择谁呢？"<|im_end|>\n<|im_start|>assistant\n阿猛可以cos大嫂吗 或者缸灿团队来cos个狂飙想看！！[星星眼][星星眼][星星眼]<|im_end|>', '<|im_start|>system\n你是一个 bilibili 或者 B 站的用户, 你的任务是回复网友的问题<|im_end|>\n<|im_start|>user\n"怎样让中年男同事瞬间变得时尚硬汉？"<|im_end|>\n<|im_start|>assistant\n建议把小缸改造成卡车硬汉：李有田[doge]<|im_end|>']
15318
[8422, 12066, 861, 10605, 3053, 10479, 10520, 6275, 6014, 1921, 13805, 12496, 242, 8916, 14951, 6610, 9215, 2104, 3079, 14073, 3930, 445, 3305, 12460, 841, 1141, 790, 12890, 4129, 7874, 5230, 4952, 7092, 14286, 8215, 11046, 8488, 5011, 14705, 14461, 10445, 10953, 9040, 9188, 4426, 11036, 7133, 10021, 9468, 192, 13542, 6140, 14438, 10504, 13874, 5634, 8427, 13264, 1499, 4152, 342, 12663, 10859, 5113, 6409, 9105, 4361, 161, 10340, 3328, 4552, 4324, 5222, 4454, 7953, 7040, 7466, 14728, 13803, 8309, 10886, 11638, 13313, 2397, 5912, 12557, 4815, 14875, 6047, 854, 10883, 14985, 5252, 7351, 1772, 13

In [3]:
print(data.column_names)

{'train': ['intstruction'], 'test': ['intstruction']}


In [4]:
model_path= '/root/autodl-tmp/qwen/Qwen2-7B-Instruct'
tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=False)
print(tokenizer.pad_token_id)
print(tokenizer.decode(tokenizer.pad_token_id))
print(tokenizer.pad_token)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


151643
<|endoftext|>
<|endoftext|>


In [5]:
def tokenize_function(examples):
    # extract text
    text = examples["intstruction"]

    #tokenize and truncate text
    tokenizer.truncation_side = "left"
    tokenized_inputs = tokenizer(
        f"{text}",
        # return_tensors="np", # return numpy array
        add_special_tokens=False
    )
    # print(text)
    # print(tokenized_inputs)
    # return {
    #     "input_ids": tokenized_inputs['input_ids'] + [tokenizer.pad_token_id],
    #     "attention_mask": tokenized_inputs['attention_mask'] + [1],
    #      "labels": tokenized_inputs['input_ids'] + [tokenizer.pad_token_id]
    # }
    return {
        "input_ids": tokenized_inputs['input_ids'] ,
        "attention_mask": tokenized_inputs['attention_mask'] ,
         "labels": tokenized_inputs['input_ids']
    }

# tokenize training and validation datasets
tokenized_data = data.map(tokenize_function, remove_columns=['intstruction'])
print(tokenized_data)
print(tokenized_data['train'][0])
tokenizer.decode(tokenized_data['train'][0]['input_ids'])

Map:   0%|          | 0/15218 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 15218
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 100
    })
})
{'input_ids': [151644, 8948, 198, 56568, 101909, 20316, 85948, 92313, 28946, 425, 10236, 104, 247, 9370, 20002, 11, 220, 103929, 88802, 20412, 104787, 100744, 103936, 151645, 198, 151644, 872, 198, 1, 99727, 100525, 3837, 102056, 30534, 9407, 46944, 100780, 3837, 102762, 50404, 100165, 101036, 11319, 1, 151645, 198, 151644, 77091, 198, 99727, 100525, 73670, 9407, 26288, 109790, 101037, 92313, 28946, 107347, 103322, 103932, 36407, 9407, 18947, 101231, 103794, 99172, 50930, 31251, 58, 109014, 99246, 1457, 109014, 99246, 1457, 109014, 99246, 60, 151645], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

'<|im_start|>system\n你是一个 bilibili 或者 B 站的用户, 你的任务是回复网友的问题<|im_end|>\n<|im_start|>user\n"阿猛，如果你要cos一个角色，你会选择谁呢？"<|im_end|>\n<|im_start|>assistant\n阿猛可以cos大嫂吗 或者缸灿团队来cos个狂飙想看！！[星星眼][星星眼][星星眼]<|im_end|>'

In [6]:
model = AutoModelForCausalLM.from_pretrained(model_path, device_map="cuda",torch_dtype=torch.bfloat16)
model.enable_input_require_grads()
model.dtype

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

torch.bfloat16

In [7]:
# lora config
config = LoraConfig(
    task_type=TaskType.CAUSAL_LM, 
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    inference_mode=False, # 训练模式
    r=8, # Lora 秩
    lora_alpha=32, # Lora alaph，具体作用参见 Lora 原理
    lora_dropout=0.1# Dropout 比例
)
print(config)


LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, r=8, target_modules={'q_proj', 'gate_proj', 'v_proj', 'k_proj', 'o_proj', 'up_proj', 'down_proj'}, lora_alpha=32, lora_dropout=0.1, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False, layer_replication=None)


In [8]:
model = get_peft_model(model, config)
model.print_trainable_parameters()

trainable params: 20,185,088 || all params: 7,635,801,600 || trainable%: 0.26434798934534914


In [9]:
# 配置训练参数
args = TrainingArguments(
    output_dir="/root/autodl-tmp/loraoutput/Qwen2_instruct_lora0715",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    logging_steps=100,
    num_train_epochs=3,
    save_steps=100, # 为了快速演示，这里设置10，建议你设置成100
    learning_rate=1e-4,
    save_on_each_node=True,
    gradient_checkpointing=True
)
# print(args)

In [10]:
print(tokenizer.decode(tokenized_data['train'][200]['input_ids']))
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_data['train'],
    eval_dataset=tokenized_data["test"],
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
)

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


<|im_start|>system
你是一个 bilibili 或者 B 站的用户, 你的任务是回复网友的问题<|im_end|>
<|im_start|>user
"如果陷入犯罪诱惑，你会如何选择，像【拳影回忆录】里的角色一样吗？"<|im_end|>
<|im_start|>assistant
一千个读者里面就有一千个哈姆雷特，个人感觉是主角和同伴在成长的路上被“犯罪”诱惑了，同伴已经迷失在白雾中，自己追上去的时候才发现自己是错的，自己想回头的时候看见同伴的背包也就是背负着的变成骂名，现在已经回不去了“罪名”将自己关进监狱了，自己看着自己的脏手很后悔，同伴也后悔莫及的看着你<|im_end|>


In [ ]:
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/root/miniconda3/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
100,2.263800
200,2.157400
300,2.132100


/root/miniconda3/lib/python3.10/site-packages/peft/utils/save_and_load.py:154: UserWarning: Could not find a config file in /root/autodl-tmp/qwen/Qwen2-7B-Instruct - will assume that the vocabulary was not modified.
  warnings.warn(
/root/miniconda3/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/root/miniconda3/lib/python3.10/site-packages/peft/utils/save_and_load.py:154: UserWarning: Could not find a config file in /root/autodl-tmp/qwen/Qwen2-7B-Instruct - will assume that the vocabulary was not modified.
  warnings.warn(
/root/miniconda3/lib/python3.10/site-packages/torch/utils/checkpoint.py:4

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from peft import PeftModel
mode_path = '/root/autodl-tmp/qwen/Qwen2-7B-Instruct/'
lora_path = '/root/autodl-tmp/loraoutput/Qwen2_instruct_lora/checkpoint-20' # 这里改称你的 lora 输出对应 checkpoint 地址

# 加载tokenizer
tokenizer = AutoTokenizer.from_pretrained(mode_path, trust_remote_code=True)

# 加载模型
model = AutoModelForCausalLM.from_pretrained(mode_path, device_map="auto",torch_dtype=torch.bfloat16, trust_remote_code=True).eval()

# 加载lora权重
model = PeftModel.from_pretrained(model, model_id=lora_path)

prompt = "你是谁？"
inputs = tokenizer.apply_chat_template([{"role": "user", "content": "假设你是皇帝身边的女人--甄嬛。"},{"role": "user", "content": prompt}],
                                       add_generation_prompt=True,
                                       tokenize=True,
                                       return_tensors="pt",
                                       return_dict=True
                                       ).to('cuda')


gen_kwargs = {"max_length": 2500, "do_sample": True, "top_k": 1}
with torch.no_grad():
    outputs = model.generate(**inputs, **gen_kwargs)
    outputs = outputs[:, inputs['input_ids'].shape[1]:]
    print(tokenizer.decode(outputs[0], skip_special_tokens=True))